### 说明
- 基于440000预训练的dense模型进行初始化，0个共享专家，8个非共享专家，每层都插入moe，共享专家直接复制dense的mlp，带有mgate，添加了router_gate，并随机初始化
- 其次，采用openmoe的实现

In [1]:
# !pip install tensorflow==2.16.1
# pip install numpy==1.26.4

import json
import os
import sys
import asyncio
import argparse
from collections import defaultdict
import time

os.environ["JAX_PLATFORMS"] = "cpu"
from etils import epath
import json
import base64

import torch
import numpy as np
import jax.numpy as jnp
import jax
import orbax
import orbax.checkpoint as ocp
from etils import epath
from jax.sharding import PartitionSpec as PS
from flax.traverse_util import flatten_dict, unflatten_dict


METADATA_FILE = '_METADATA'
_CHECKPOINT_FILE = 'checkpoint'


read_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/PileDCSlimLlama7B32Kx4x256x1v5p_0713/checkpoints/440000/state'
save_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_base440k_1127/checkpoints'

read_dir = epath.Path(read_dir) 
save_dir = epath.Path(save_dir)

metadata_path = read_dir / METADATA_FILE
back_metadata_path = read_dir / f'{METADATA_FILE}.back'
try:
    metadata_path.rename(back_metadata_path)
except:
    pass
metadata_path.unlink(missing_ok=True) # delete
structure_path = read_dir / _CHECKPOINT_FILE
msgpack = ocp.aggregate_handlers.MsgpackHandler(0)
structure = msgpack.deserialize(structure_path)
# backup original checkpoint fil
back_structure_path = read_dir / 'checkpoint_back'
back_structure = structure.copy()
if not back_structure_path.exists():
    asyncio.run(msgpack.serialize(back_structure_path, item=back_structure))
print(f'Old structure file keys: {structure.keys()}')
remove_keys = ['opt_state', 'step'] # select the weight name you don't want to load, all weight name: opt_state, step, params
_ = [structure.pop(key) for key in remove_keys if key in structure]
print(f'New structure file keys: {structure.keys()}')
asyncio.run(msgpack.serialize(structure_path, item=structure))  # rewrite struct file

# load model based struct, note: axes must same as training
mesh_axes = ['data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'tensor', 'autoregressive']
devices = np.asarray(jax.devices()).reshape([1] * len(mesh_axes))
mesh = jax.sharding.Mesh(devices, mesh_axes)
sharding = jax.sharding.NamedSharding(mesh, PS()) # Sharding is None because we use cpu to load weights
weight_dtype = jnp.bfloat16 # set restore weights dtype
restore_args = {}
for k, v in flatten_dict(structure).items():
    restore_args[k] =  ocp.ArrayRestoreArgs(restore_type=jax.Array, dtype=weight_dtype, sharding=sharding)
restore_args = unflatten_dict(restore_args)
ckptr = ocp.Checkpointer(ocp.PyTreeCheckpointHandler())
w = ckptr.restore(read_dir, args=ocp.args.PyTreeRestore(restore_args=restore_args))
structure_path = read_dir / _CHECKPOINT_FILE
# rewrite struct file, otherwise occur error when continue training
asyncio.run(msgpack.serialize(structure_path, item=back_structure))
while 'params' in w:
    w = w['params']
xm3p5_w = {'.'.join(k): np.array(v) for k, v in flatten_dict(w).items()}

try:
    back_metadata_path.rename(metadata_path)
except:
    pass

2024-11-27 05:21:24.380146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 05:21:24.393807: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 05:21:24.393832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 05:21:26.564557: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Old structure file keys: dict_keys(['params'])
New structure file keys: dict_keys(['params'])


I0000 00:00:1732684894.079427   14637 gcs_resource.cc:109] Using default AdmissionQueue with limit 32
I0000 00:00:1732684894.083712   15164 google_auth_provider.cc:180] Running on GCE, using service account 887571727717-compute@developer.gserviceaccount.com


In [65]:
for k, v in xm3p5_w.items():
    print(k, v.shape)

decoder.decoder_norm.scale (4096,)
decoder.layers.mlp_0.mgate.kernel (4096, 12, 44)
decoder.layers.mlp_0.wi_0.kernel (4096, 12, 5632)
decoder.layers.mlp_0.wi_1.kernel (4096, 12, 5632)
decoder.layers.mlp_0.wo.kernel (5632, 12, 4096)
decoder.layers.mlp_1.mgate.kernel (4096, 12, 44)
decoder.layers.mlp_1.wi_0.kernel (4096, 12, 5632)
decoder.layers.mlp_1.wi_1.kernel (4096, 12, 5632)
decoder.layers.mlp_1.wo.kernel (5632, 12, 4096)
decoder.layers.mlp_2.mgate.kernel (4096, 12, 44)
decoder.layers.mlp_2.wi_0.kernel (4096, 12, 5632)
decoder.layers.mlp_2.wi_1.kernel (4096, 12, 5632)
decoder.layers.mlp_2.wo.kernel (5632, 12, 4096)
decoder.layers.mlp_3.mgate.kernel (4096, 12, 44)
decoder.layers.mlp_3.wi_0.kernel (4096, 12, 5632)
decoder.layers.mlp_3.wi_1.kernel (4096, 12, 5632)
decoder.layers.mlp_3.wo.kernel (5632, 12, 4096)
decoder.layers.post_self_attention_layer_norm_0.scale (4096, 12)
decoder.layers.post_self_attention_layer_norm_1.scale (4096, 12)
decoder.layers.post_self_attention_layer_norm_2

In [11]:

## 基于dense模型保存和moe相同名字的参数
def convert_to_jnp(params, remove_keys=[]):
    convert_params = {}
    for k, v in params.items():
        r = 0
        for remove_key in remove_keys:
            if remove_key in k: 
                r = 1
                break
        if r: continue
        k = tuple(k.split('.'))
        convert_params[k] = v
        # convert_params[k] = jnp.array(v).astype(jnp.bfloat16)
    for k, v in convert_params.items():
        print(k, v.shape, v.dtype)
    return convert_params

def save_params(step, save_dir, params):
    item = {
            'state': orbax.checkpoint.AsyncCheckpointer(orbax.checkpoint.PyTreeCheckpointHandler(use_ocdbt=False)),
                    }
    # new_params = {tuple(k.split('.')): v for k,v in params.items()}
    unflatten_params = unflatten_dict(params)
    for k, v in params.items():
        print(k, v.shape, v.dtype)
    mngr = orbax.checkpoint.CheckpointManager(save_dir, item)
    if 'params' not in unflatten_params: unflatten_params = {'params': unflatten_params}
    mngr.save(step, items={'state': {'params': unflatten_params}})
remove_keys = ['mlp_']
convert_params = convert_to_jnp(xm3p5_w, remove_keys)
save_params(0, save_dir, convert_params)

('decoder', 'decoder_norm', 'scale') (4096,) bfloat16
('decoder', 'layers', 'post_self_attention_layer_norm_0', 'scale') (4096, 12) bfloat16
('decoder', 'layers', 'post_self_attention_layer_norm_1', 'scale') (4096, 12) bfloat16
('decoder', 'layers', 'post_self_attention_layer_norm_2', 'scale') (4096, 12) bfloat16
('decoder', 'layers', 'post_self_attention_layer_norm_3', 'scale') (4096, 12) bfloat16
('decoder', 'layers', 'pre_self_attention_layer_norm_0', 'scale') (4096, 12) bfloat16
('decoder', 'layers', 'pre_self_attention_layer_norm_1', 'scale') (4096, 12) bfloat16
('decoder', 'layers', 'pre_self_attention_layer_norm_2', 'scale') (4096, 12) bfloat16
('decoder', 'layers', 'pre_self_attention_layer_norm_3', 'scale') (4096, 12) bfloat16
('decoder', 'layers', 'self_attention_0', 'AttentionOp_0', 'dyn_w_proj', 'dd', 'kernel') (4096, 12, 1, 128) bfloat16
('decoder', 'layers', 'self_attention_0', 'AttentionOp_0', 'dyn_w_proj', 'dw1', 'kernel') (4096, 12, 1, 4, 128) bfloat16
('decoder', 'lay

In [106]:
def nd_dense_init(scale, mode, distribution):
  """Initializer with in_axis, out_axis set at call time."""

  def init_fn(key, shape, dtype, in_axis, out_axis):
    fn = jax.nn.initializers.variance_scaling(scale, mode, distribution, in_axis, out_axis)
    return fn(key, shape, dtype)

  return init_fn

init_func = nd_dense_init(1.0, "fan_in", "truncated_normal")


## moe部分的参数保存，保存后在bucket后台人工进行转移
start_time = time.time()
unshared_experts = 8

scale = 1
mlp_dim = 5632 // scale
model_dim = 4096 // scale
copy_dim = 5632 // scale
# 4个子层
total_moe_params = [{} for i in range(4)]
gate_seed = 9876

for k, v in xm3p5_w.items():
    v = jnp.array(v).astype(jnp.bfloat16)
    print(f'k: {k} take: {time.time() - start_time:.3f}s')
    if 'decoder.layers.mlp_' in k:
        mlp_inx = k.find('mlp_')
        l = k[mlp_inx+4: mlp_inx+5]
        # if int(l) % 2 != 0: continue
        moe_params = total_moe_params[int(l)]
        unshared_mlp = k.replace('decoder.layers.mlp_', 'decoder.layers.unshared_mlp_')
        if 'mgate' in unshared_mlp:
            unshared_mlp1 = unshared_mlp.replace('.mgate', '.router_gate')
            gate_seed += 1
            init_v = init_func(key=jax.random.PRNGKey(gate_seed), shape=[*v.shape[:-1], unshared_experts], dtype=jnp.bfloat16, in_axis=0, out_axis=1)
            moe_params[unshared_mlp1] = init_v
        
        unshared_mlp = unshared_mlp.replace('.kernel', '')
        v = v.transpose(1, 0, 2)
        # unshared: 44 * 12 * model_dim * mlp_dim,  mlp: model_dim * 12 * mlp_dim
        copy_w = v[:, jnp.newaxis].repeat(unshared_experts, 1).transpose(1, 0, 2, 3)
        init_w = copy_w
        print(f'copy_w: {copy_w.shape} copy_w: {copy_w.dtype} init_w: {init_w.dtype} unshared_mlp: {unshared_mlp} v: {v.shape}\n\n')
        moe_params[unshared_mlp] = init_w


k: decoder.decoder_norm.scale take: 0.001s
k: decoder.layers.mlp_0.mgate.kernel take: 0.001s
k: decoder.layers.mlp_0.wi_0.kernel take: 0.221s
k: decoder.layers.mlp_0.wi_1.kernel take: 0.228s
k: decoder.layers.mlp_0.wo.kernel take: 0.234s
k: decoder.layers.mlp_1.mgate.kernel take: 0.234s
k: decoder.layers.mlp_1.wi_0.kernel take: 0.235s
k: decoder.layers.mlp_1.wi_1.kernel take: 0.235s
k: decoder.layers.mlp_1.wo.kernel take: 0.235s
k: decoder.layers.mlp_2.mgate.kernel take: 0.235s
k: decoder.layers.mlp_2.wi_0.kernel take: 0.235s
k: decoder.layers.mlp_2.wi_1.kernel take: 0.236s
k: decoder.layers.mlp_2.wo.kernel take: 0.236s
k: decoder.layers.mlp_3.mgate.kernel take: 0.236s
k: decoder.layers.mlp_3.wi_0.kernel take: 0.236s
k: decoder.layers.mlp_3.wi_1.kernel take: 0.236s
k: decoder.layers.mlp_3.wo.kernel take: 0.236s
k: decoder.layers.post_self_attention_layer_norm_0.scale take: 0.236s
k: decoder.layers.post_self_attention_layer_norm_1.scale take: 0.236s
k: decoder.layers.post_self_attention

In [107]:
for i, params in enumerate(total_moe_params):
    if not params: continue
    save_moe_params = {}
    print(f'\n\nSave step: {i+1}')
    for k, v in params.items():
        newk = tuple(k.split('.'))
        save_moe_params[newk] = v
        print(k, v.shape)
    save_params(i + 1, save_dir, save_moe_params)



Save step: 1
decoder.layers.unshared_mlp_0.router_gate.kernel (4096, 12, 8)
('decoder', 'layers', 'unshared_mlp_0', 'router_gate', 'kernel') (4096, 12, 8) bfloat16




Save step: 2
decoder.layers.unshared_mlp_1.router_gate.kernel (4096, 12, 8)
('decoder', 'layers', 'unshared_mlp_1', 'router_gate', 'kernel') (4096, 12, 8) bfloat16




Save step: 3
decoder.layers.unshared_mlp_2.router_gate.kernel (4096, 12, 8)
('decoder', 'layers', 'unshared_mlp_2', 'router_gate', 'kernel') (4096, 12, 8) bfloat16




Save step: 4
decoder.layers.unshared_mlp_3.router_gate.kernel (4096, 12, 8)
('decoder', 'layers', 'unshared_mlp_3', 'router_gate', 'kernel') (4096, 12, 8) bfloat16


In [95]:
# _METADATA, checkpoint三个文件转移到实际保存的checkpoint文件夹中进行替换
start_time = time.time()
moe_params = {}
flag = 0
for k, v in xm3p5_w.items():
    print(f'k: {k} take: {time.time() - start_time:.3f}s')
    if 'decoder.layers.mlp_' in k:
        mlp_inx = k.find('mlp_')
        l = k[mlp_inx+4: mlp_inx+5]
        # mgate需要.kernel, moe不需要，因为moe的参数采用的是self.params创建的，而mgate是采用flax的DenseGeneral
        unshared_mlp = k.replace('decoder.layers.mlp_', 'decoder.layers.unshared_mlp_')
        if 'mgate' in unshared_mlp:
            unshared_mlp1 = unshared_mlp.replace('.mgate', '.router_gate')
            unshared_mlp2 = unshared_mlp.replace('.kernel', '')
            moe_params[unshared_mlp1] = v
            moe_params[unshared_mlp2] = v
        else:
            unshared_mlp = unshared_mlp.replace('.kernel', '')
            moe_params[unshared_mlp] = v
    else:
        moe_params[k] = v

moe_params = {tuple(k.split('.')): v for k,v in moe_params.items()}
save_params(5, save_dir, moe_params)

k: decoder.decoder_norm.scale take: 0.000s
k: decoder.layers.mlp_0.mgate.kernel take: 0.000s
k: decoder.layers.mlp_0.wi_0.kernel take: 0.000s
k: decoder.layers.mlp_0.wi_1.kernel take: 0.000s
k: decoder.layers.mlp_0.wo.kernel take: 0.000s
k: decoder.layers.mlp_1.mgate.kernel take: 0.000s
k: decoder.layers.mlp_1.wi_0.kernel take: 0.000s
k: decoder.layers.mlp_1.wi_1.kernel take: 0.000s
k: decoder.layers.mlp_1.wo.kernel take: 0.000s
k: decoder.layers.mlp_2.mgate.kernel take: 0.000s
k: decoder.layers.mlp_2.wi_0.kernel take: 0.000s
k: decoder.layers.mlp_2.wi_1.kernel take: 0.000s
k: decoder.layers.mlp_2.wo.kernel take: 0.000s
k: decoder.layers.mlp_3.mgate.kernel take: 0.000s
k: decoder.layers.mlp_3.wi_0.kernel take: 0.000s
k: decoder.layers.mlp_3.wi_1.kernel take: 0.000s
k: decoder.layers.mlp_3.wo.kernel take: 0.000s
k: decoder.layers.post_self_attention_layer_norm_0.scale take: 0.000s
k: decoder.layers.post_self_attention_layer_norm_1.scale take: 0.000s
k: decoder.layers.post_self_attention

In [102]:
for k, v in moe_params.items():
    print(k)

('decoder', 'decoder_norm', 'scale')
('decoder', 'layers', 'unshared_mlp_0', 'router_gate', 'kernel')
('decoder', 'layers', 'unshared_mlp_0', 'mgate')
('decoder', 'layers', 'unshared_mlp_0', 'wi_0')
('decoder', 'layers', 'unshared_mlp_0', 'wi_1')
('decoder', 'layers', 'unshared_mlp_0', 'wo')
('decoder', 'layers', 'unshared_mlp_1', 'router_gate', 'kernel')
('decoder', 'layers', 'unshared_mlp_1', 'mgate')
('decoder', 'layers', 'unshared_mlp_1', 'wi_0')
('decoder', 'layers', 'unshared_mlp_1', 'wi_1')
('decoder', 'layers', 'unshared_mlp_1', 'wo')
('decoder', 'layers', 'unshared_mlp_2', 'router_gate', 'kernel')
('decoder', 'layers', 'unshared_mlp_2', 'mgate')
('decoder', 'layers', 'unshared_mlp_2', 'wi_0')
('decoder', 'layers', 'unshared_mlp_2', 'wi_1')
('decoder', 'layers', 'unshared_mlp_2', 'wo')
('decoder', 'layers', 'unshared_mlp_3', 'router_gate', 'kernel')
('decoder', 'layers', 'unshared_mlp_3', 'mgate')
('decoder', 'layers', 'unshared_mlp_3', 'wi_0')
('decoder', 'layers', 'unshared_m

In [96]:
import subprocess

# save_dir1 = str(save_dir).rstrip('/')
source_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_base440k_1127/checkpoints'
target_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_base440k_1127/checkpoints'

source_step = 5
command = f'gsutil cp {source_dir}/{source_step}/state/_METADATA {target_dir}/0/state/ '
r = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

command = f'gsutil cp {source_dir}/{source_step}/state/checkpoint {target_dir}/0/state/ '
r = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

# command = f'gsutil cp {source_dir}/{source_step}/state/_sharding {target_dir}/0/state/ '
# r = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

Copying gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_base440k_1127/checkpoints/5/state/_METADATA [Content-Type=application/octet-stream]...
/ [1 files][ 27.8 KiB/ 27.8 KiB]                                                
Operation completed over 1 objects/27.8 KiB.                                     
Copying gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_base440k_1127/checkpoints/5/state/checkpoint...
/ [1 files][  6.2 KiB/  6.2 KiB]                                                
Operation completed over 1 objects/6.2 KiB.                                      


In [101]:
# # 基于tpu type 构建_sharding文件

# import base64

# def decode_base64(encoded_str):
#     decoded_bytes = base64.b64decode(encoded_str)
#     decoded_str = decoded_bytes.decode('utf-8')
#     return decoded_str


# '''
# _sharding文件格式如下：
# {
#   b3B0X3N0YXRlLm11LnBhcmFtcy50b2tlbl9lbWJlZGRlci5lbWJlZGRpbmc=': {'sharding_type': 'NamedSharding',
#   'shape': [1, 1, 4, 1, 1, 1, 1],
#   'axis_names': ['data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'tensor','autoregressive'],
#   'partition_spec': [['tensor', 'autoregressive'], ['fsdp', 'fsdp_transpose', 'sequence']],
#    2: 4},
#    ...
#    }
#    '''
# # moe sharding
# _sharding_path1 = 'gs://llm_base_models_us-east5/v5p_256/7B/xm_45x7B_moe_1017/xm3p5_moe_params_no_opt_v5p_64_sharding.copy'
# _sharding_path2 = 'gs://llm_base_models_europe-west4/v5p_256/7B/_sharding_per_layer_moe'

# _sharding_path1 = epath.Path(_sharding_path1)
# _sharding_path2 = epath.Path(_sharding_path2)

# # 读取已有的_sharding文件
# with _sharding_path1.open('r') as f:
#     _sharding1 = json.load(f)

# with _sharding_path2.open('r') as f:
#     _sharding2 = json.load(f)
    
# tpu_type = 'v5p-64'
# core_nums = int(tpu_type.split('-')[-1])
# if 'v3' not in tpu_type:
#     core_nums = core_nums // 2
# print(f'core_nums: {core_nums}')
# updated_sharding2 = {}
# base_ks = []
# for k, v in _sharding2.items():
#     base_k = decode_base64(k)
#     if 'opt_state' in base_k: continue
    
#     if 'mgate' in base_k or 'router' in base_k:
#         print(base_k)
#         print(v)
#         print('\n\n')
#     # print(base_k)
#     base_ks.append(base_k)
#     v = json.loads(v)
#     v['shape'][2] = core_nums
#     updated_sharding2[base_k] = k


# updated_sharding = {}
# base_ks = []
# for k, v in _sharding1.items():
#     base_k = decode_base64(k)
#     if 'opt_state' in base_k: continue
#     if 'layers.mlp_' in base_k: continue
  
#     base_ks.append(base_k)
#     v = json.loads(v)
#     v['shape'][2] = core_nums
#     updated_sharding[k] = json.dumps(v)

#     if 'unshared_mlp_0' in base_k:
#         unshared_mlp_1 = base_k.replace('unshared_mlp_0', 'unshared_mlp_1')
#         unshared_mlp_3 = base_k.replace('unshared_mlp_0', 'unshared_mlp_3')
#         encode_mlp_1 = updated_sharding2[unshared_mlp_1]
#         encode_mlp_3 = updated_sharding2[unshared_mlp_3]
        
#         updated_sharding[encode_mlp_1] = json.dumps(v)
#         updated_sharding[encode_mlp_3] = json.dumps(v)
#         base_ks.append(encode_mlp_1)
#         base_ks.append(encode_mlp_3)
    
# updated_sharding_path = f'{save_dir}/0/state/_sharding'

# updated_sharding_path = epath.Path(updated_sharding_path)
# with updated_sharding_path.open('w') as f:
#     json.dump(updated_sharding, f)

# print(f'length: {len(base_ks)}')

In [99]:
# 基于tpu type 构建_sharding文件

import base64

def decode_base64(encoded_str):
    decoded_bytes = base64.b64decode(encoded_str)
    decoded_str = decoded_bytes.decode('utf-8')
    return decoded_str


'''
_sharding文件格式如下：
{
  b3B0X3N0YXRlLm11LnBhcmFtcy50b2tlbl9lbWJlZGRlci5lbWJlZGRpbmc=': {'sharding_type': 'NamedSharding',
  'shape': [1, 1, 4, 1, 1, 1, 1],
  'axis_names': ['data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'tensor','autoregressive'],
  'partition_spec': [['tensor', 'autoregressive'], ['fsdp', 'fsdp_transpose', 'sequence']],
   2: 4},
   ...
   }
   '''
# moe sharding
# _sharding_path1 = 'gs://llm_base_models_us-east5/v5p_256/7B/xm_45x7B_moe_1017/xm3p5_moe_params_no_opt_v5p_64_sharding.copy'
_sharding_path2 = 'gs://llm_base_models_europe-west4/v5p_256/7B/_sharding_per_layer_moe'
_sharding_path = epath.Path(_sharding_path)
# 读取已有的_sharding文件
with _sharding_path.open('r') as f:
    _sharding = json.load(f)

tpu_type = 'v5p-64'
core_nums = int(tpu_type.split('-')[-1])
if 'v3' not in tpu_type:
    core_nums = core_nums // 2
print(f'core_nums: {core_nums}')
updated_sharding = {}
for k, v in _sharding.items():
    base_k = decode_base64(k)
    if 'opt_state' in base_k or 'step' in base_k: continue
    # if 'unshared_mlp' in base_k and 'mgate' in base_k:
    #     continue
    v = json.loads(v)
    v['shape'][2] = core_nums
    updated_sharding[k] = json.dumps(v)
    
updated_sharding_path = f'{save_dir}/0/state/_sharding'

updated_sharding_path = epath.Path(updated_sharding_path)
with updated_sharding_path.open('w') as f:
    json.dump(updated_sharding, f)

core_nums: 32


In [100]:
len(updated_sharding)

67

In [103]:
for k, v in updated_sharding.items():
    base_k = decode_base64(k)
    print(base_k)

params.params.token_embedder.embedding
params.params.decoder.decoder_norm.scale
params.params.decoder.layers.unshared_mlp_0.wi_0
params.params.decoder.layers.unshared_mlp_0.wi_1
params.params.decoder.layers.unshared_mlp_0.wo
params.params.decoder.layers.unshared_mlp_0.mgate
params.params.decoder.layers.unshared_mlp_0.router_gate.kernel
params.params.decoder.layers.unshared_mlp_1.wi_0
params.params.decoder.layers.unshared_mlp_1.wi_1
params.params.decoder.layers.unshared_mlp_1.wo
params.params.decoder.layers.unshared_mlp_1.mgate
params.params.decoder.layers.unshared_mlp_1.router_gate.kernel
params.params.decoder.layers.unshared_mlp_2.wi_0
params.params.decoder.layers.unshared_mlp_2.wi_1
params.params.decoder.layers.unshared_mlp_2.wo
params.params.decoder.layers.unshared_mlp_2.mgate
params.params.decoder.layers.unshared_mlp_2.router_gate.kernel
params.params.decoder.layers.unshared_mlp_3.wi_0
params.params.decoder.layers.unshared_mlp_3.wi_1
params.params.decoder.layers.unshared_mlp_3.wo
p